In [ ]:
!sudo apt install swig cmake

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

In [ ]:
!sudo apt-get update
!sudo apt install -y python-opengl ffmpeg xvfb 
!pip3 install pyvirtualdisplay

In [42]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [8]:

import gymnasium as gym
from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [10]:
env = gym.make("LunarLander-v2")

observation, info = env.reset()

for _ in range(20):
    action = env.action_space.sample()
    print(f"using action {action}")
    observation, reward, stopped, truned, info = env.step(action)
    if stopped or truned:
        print(f"will reset")
        observation, info = env.reset()

env.close()


using action 1
using action 1
using action 1
using action 3
using action 2
using action 2
using action 1
using action 2
using action 0
using action 1
using action 0
using action 1
using action 1
using action 3
using action 0
using action 0
using action 1
using action 1
using action 0
using action 2


In [26]:
print(env.observation_space.shape)
print(env.observation_space.sample())

(8,)
[ 19.037996   -21.286295     3.0375197   -2.3604877   -1.8046252
   4.0636477    0.7060533    0.02476957]


In [22]:
print(env.action_space.n)
print(env.action_space.sample())

4
2


In [30]:
env2 = gym.make_vec("LunarLander-v2", num_envs=16)



In [36]:
# try PPO with LunarLander

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps= int(2e5))

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 89.2     |
|    ep_rew_mean     | -176     |
| time/              |          |
|    fps             | 106      |
|    iterations      | 1        |
|    time_elapsed    | 19       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 92.4        |
|    ep_rew_mean          | -171        |
| time/                   |             |
|    fps                  | 161         |
|    iterations           | 2           |
|    time_elapsed         | 25          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008146513 |
|    clip_fraction        | 0.0229      |
|    clip_range           | 0.2         |
|    entropy_loss  

In [37]:
model.save("lz_1_ppo")

In [38]:

# eval it
from stable_baselines3.common.evaluation import evaluate_policy

e_env = gym.make("LunarLander-v2", render_mode="rgb_array")

model = PPO.load("lz_1_ppo", env=e_env)

mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

print(mean_reward, std_reward)



Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
76.3280828 136.91123779397464


In [43]:
v_env = model.get_env()
obs = v_env.reset()
for i in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, dones, info = v_env.step(action)
    v_env.render("human")

In [47]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env


from huggingface_sb3 import package_to_hub

# PLACE the variables you've just defined two cells above
# Define the name of the environment
env_id = "LunarLander-v2"

model_name = 'lz_1'
# TODO: Define the model architecture we used
model_architecture = "PPO"

## Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
## CHANGE WITH YOUR REPO ID
repo_id = "ThomasSimonini/ppo-LunarLander-v2" # Change with your repo id, you can't push with mine 😄

## Define the commit message
commit_message = "Upload PPO LunarLander-v2 trained agent"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])

# PLACE the package_to_hub function you've just filled here
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model 
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.